In [1]:
###load packages
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
from http import cookiejar
from time import sleep
import random
#from lxml import html
#import lxml
import re

import requests_cache
requests_cache.install_cache('coursedetail')

The goal of this book is to scrapt detail information of each course (eg. max number, location, time .etc)

In [3]:
###get dept
def getdept():
    url = 'http://ucdavisstores.com/SelectTermDept/Department'
    headers = {'User-Agent': "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) Gecko/20100101 Firefox/40.1",
           'X-Requested-With': 'XMLHttpRequest',
           'Host': 'ucdavisstores.com',
           'Referer': 'http://ucdavisstores.com/SelectTermDept/Courses'} 
    params = {'termId': '8247'}
    response = requests.post(url, params=params, headers=headers)
    return(response)
###get the dataframe of department name and its id
dept = pd.DataFrame(columns=['did',"dname"])
response = getdept()
bs = BeautifulSoup(response.json(),"lxml")
i = 0
for ce in bs.findAll("li",{"class":"ordering_hover_color ordering_enabled padding10 hover_pointer bottom_border list_display"}):
    deptid = ce.attrs["data-id"]
    deptname = ce.get_text()
    dept.loc[i]={'did':deptid,"dname":deptname}
    i = i + 1
dept.head()

,did,dname
0,dpt-23459,AAS
1,dpt-25725,ABG
2,dpt-23462,ABI
3,dpt-23463,ABT
4,dpt-38037,ACC


In [26]:
####### find all the courses and their CRN opened by a certain department #######
def CourseByDept(dept,quarter="201801"):
    session = requests.session()
    data = {"termCode":quarter,"course_number":dept}
    print(quarter)
    print(dept)
    courseCRN = "https://registrar.ucdavis.edu/courses/search/course_search_results.cfm"
    header = {
    'User-Agent': "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) Gecko/20100101 Firefox/40.1"
}
    response = session.post(courseCRN, data=data,headers=header)
    bs = BeautifulSoup(response.text,"lxml")
    table = bs.find("table",{"id":"mc_win"})
    records = []
    if table != None:
        for tr in table.findAll("tr"):
            
            trs = tr.findAll("td")
            record = []
            try:
                record.append(trs[0].text.strip())
                record.append(trs[1].text.strip())
                record.append(trs[2].text.strip())
                record.append(trs[3].text.strip())
                record.append(trs[4].text.strip())
                records.append(record)
            except IndexError:
                pass
        df = pd.DataFrame(data=records)
        df.columns = ['C_T', 'C_L', 'S','T_PG_NG','I_U']
        df = df.drop(0)
        df["C_T"] = df["C_T"].str.replace(r'\s{3,6}', r' ')
        df["S"] = df["S"].str.replace(r'\s{1,3}', r' ')
        df["T_PG_NG"] = df["T_PG_NG"].str.replace(r'\s{2,}', r'/')
        df["I_U"] = df["I_U"].str.replace(r'\s{2,}', r' ')
        df = df.join(df['C_T'].str.split(' ', 1, expand=True).rename(columns={0:'CRN', 1:'Time'}))
        df = df.join(df['C_L'].str.split('\n', 1, expand=True).rename(columns={0:'Course', 1:'Location'}))
        course_CRN = df[["CRN","Course"]]
        return(course_CRN)

In [28]:
dataset = pd.DataFrame(columns = ['CRN','Course'])
for d in dept.dname:
    sleep(random.randrange(1,5))
    temp = CourseByDept(d,quarter="201801")
    dataset = pd.concat([dataset,temp],ignore_index=True,axis=0)

201801
AAS       
201801
ABG       
201801
ABI       
201801
ABT       
201801
ACC       
201801
AED       
201801
AGC       
201801
AHI       
201801
AMS       
201801
ANB       
201801
ANG       
201801
ANS       
201801
ANT       
201801
ARB       
201801
ARE       
201801
ART       
201801
ASA       
201801
AST       
201801
ATM       
201801
AVS       
201801
BAX       
201801
BIM       
201801
BIS       
201801
BIT       
201801
BPH       
201801
BST       
201801
CDM       
201801
CHA       
201801
CHE       
201801
CHI       
201801
CHN       
201801
CLA       
201801
CMN       
201801
CNS       
201801
COM       
201801
CRD       
201801
CRI       
201801
CST       
201801
CTS       
201801
DES       
201801
DRA       
201801
EAD       
201801
EAE       
201801
EAP       
201801
EAS       
201801
EBS       
201801
ECH       
201801
ECI       
201801
ECL       
201801
ECN       
201801
ECS       
201801
EDU       
201801
EEC       
201801
EME       
201801
EMS       
201801
ENG

In [31]:
dataset.to_csv("course_CRN.csv")

In [5]:
dataset = dataset[["CRN","Course"]]
dataset.head()

,CRN,Course
0,74279,AAS 015
1,74279,AAS 015
2,40181,AAS 016
3,74278,AAS 017
4,74280,AAS 050


In [113]:
##### scrapt key information of a course in the course page#############
def find_detail(CRN):
    url_detail = "https://registrar.ucdavis.edu/courses/search/course.cfm"
    key ={"crn":str(CRN),
            "termCode":"201801",
            "winName":"detialWin",
            "_cf_containerId":"detailWin_body",
            "_cf_nodebug":"true",
            "_cf_nocache":"true",
            "_cf_rc":"0"}
    html = requests.get(url_detail,params=key)
    text = html.text
    bs3 = BeautifulSoup(text,"lxml")

    table = bs3.findAll("table")[0]
    records = []
    if table != None:
        record = []
        for tr in table.findAll("tr"):
            trs = tr.findAll("td")
            record.append(trs[0].text.strip())
    table = bs3.findAll("table")[2]
    if table != None:
        for tr in table.findAll("tr"):
            trs = tr.findAll("td")
            record.append(trs[0].text.strip())
        records.append(record)
    table = bs3.findAll("table")[1]
    l = len(table.findAll("tr")) - 1
    df = pd.DataFrame(data=records*l)
    try:
        df.columns = ['title', 'subject', 'term','CRN','Instructor','unit','Form_GE',
                        'New_GE','Available','Maximum_seat',"Final"]
    except ValueError:

        a = bs3.findAll("table")[0].find("strong",text=re.compile("Former GE Credit"))
        b = bs3.findAll("table")[0].find("strong",text=re.compile("New GE Credit"))
        print(a)
        print(b)
        if (a == None) and (b != None):
            print(1)
            df.columns = ['title', 'subject', 'term','CRN','Instructor','unit',"New_GE",
                      'Available','Maximum_seat',"Final"]
            df.insert(1,'Form_GE',"-"*l)
        elif (a == None) and (b == None):
            print(0)
            df.columns = ['title', 'subject', 'term','CRN','Instructor','unit',
                      'Available','Maximum_seat',"Final"]
            df.insert(1,'Form_GE',"-"*l)
            df.insert(1,'New_GE',"-"*l)
           
    if table != None:
        tm = []
        for i in range(1,len(table.findAll("tr"))):
            s = []
            tr = table.findAll("tr")[i]
            trs = tr.findAll("td")
            s.append(trs[0].text.strip())
            s.append(trs[1].text.strip())
            s.append(trs[2].text.strip())
            tm.append(s)
        df.insert(1,'L_T',tm)
    return(df)

In [115]:
cou_detail = pd.DataFrame(columns=["title","L_T","subject","term","CRN","Instructor","unit",
                          "Form_GE","Ner_GE","Available","Maximum_seat","Final"])
for index,row in dataset.iterrows():
    if row.CRN != "@":
        sleep(random.randrange(1,5))
        print(row.CRN)
        dfnew = find_detail(row.CRN)
        cou_detail = pd.concat([cou_detail,dfnew],ignore_index=True,axis=0)

74279
74279
40181
74278
74280
74280
74281
74281
74283
74283
74284
74284
40192
40192
40193
40193
40194
40194
40195
40195
74286
74285
74287
40201
40201
40202
40202
40203
40203
40204
40204
75187
None
None
0
74930
None
None
0
40252
None
None
0
40253
None
None
0
40254
None
None
0
40255
None
None
0
40256
None
None
0
40371
None
None
0
40384
None
None
0
40403
None
None
0
40403
None
None
0
40404
None
None
0
40404
None
None
0
40507
None
<strong>New GE Credit (Start Fall 2011 catalog rights):</strong>
1
40507
None
<strong>New GE Credit (Start Fall 2011 catalog rights):</strong>
1
40508
None
<strong>New GE Credit (Start Fall 2011 catalog rights):</strong>
1
40508
None
<strong>New GE Credit (Start Fall 2011 catalog rights):</strong>
1
40509
None
<strong>New GE Credit (Start Fall 2011 catalog rights):</strong>
1
40509
None
<strong>New GE Credit (Start Fall 2011 catalog rights):</strong>
1
40510
None
<strong>New GE Credit (Start Fall 2011 catalog rights):</strong>
1
40510
None
<strong>New GE Credit (

45421
74170
74170
74171
74171
45422
45422
45422
45423
45423
45423
45424
45424
45424
45425
45425
45425
45426
45426
45426
45427
45427
45427
45428
45428
45428
45429
45429
45429
45430
45430
45430
45431
45431
45431
45432
45432
45432
45433
45433
45433
45434
45434
45434
45435
45435
45435
45436
45436
45436
45437
45437
45437
45438
45438
45438
45439
45439
45439
45440
45440
45440
45441
45441
45441
45442
45442
45442
45443
45443
45443
45444
45444
45444
45445
45445
45445
45446
45446
45446
45447
45447
45447
45448
45448
45448
45449
45449
45449
45450
45450
45450
45451
45451
45451
45452
45452
45452
45453
45453
45453
45454
45454
45454
45455
45455
45455
74640
74640
74640
74641
74641
74641
74642
74642
74642
74643
74643
74643
45456
45456
45456
45457
45457
45457
45458
45458
45458
45459
45459
45459
45460
45460
45460
45461
45461
45461
45462
45462
45462
45463
45463
45463
45464
45464
45464
45465
45465
45465
45466
45466
45466
45467
45467
45467
45468
45468
45468
45469
45469
45469
45470
45470
45470
45471
45471
4547

47737
None
<strong>New GE Credit (Start Fall 2011 catalog rights):</strong>
1
47737
None
<strong>New GE Credit (Start Fall 2011 catalog rights):</strong>
1
47738
None
<strong>New GE Credit (Start Fall 2011 catalog rights):</strong>
1
47738
None
<strong>New GE Credit (Start Fall 2011 catalog rights):</strong>
1
47739
None
<strong>New GE Credit (Start Fall 2011 catalog rights):</strong>
1
47739
None
<strong>New GE Credit (Start Fall 2011 catalog rights):</strong>
1
47740
None
<strong>New GE Credit (Start Fall 2011 catalog rights):</strong>
1
47740
None
<strong>New GE Credit (Start Fall 2011 catalog rights):</strong>
1
47741
None
<strong>New GE Credit (Start Fall 2011 catalog rights):</strong>
1
47741
None
<strong>New GE Credit (Start Fall 2011 catalog rights):</strong>
1
75118
None
<strong>New GE Credit (Start Fall 2011 catalog rights):</strong>
1
75118
None
<strong>New GE Credit (Start Fall 2011 catalog rights):</strong>
1
75118
None
<strong>New GE Credit (Start Fall 2011 catalog rights

75032
None
None
0
50864
None
None
0
50933
50933
50934
50934
50935
50935
50936
50936
50937
50937
50938
50938
50939
50939
50940
50940
50941
50941
50942
50942
50943
50943
50944
50944
50945
50945
50946
50946
50947
50947
50948
50948
50949
50949
50950
50950
50951
50951
50952
50952
50953
50953
50954
50954
50955
50955
50956
50956
74705
74705
74706
74706
74707
74707
74708
74708
50957
50957
50958
50958
50959
50959
50960
50960
50961
50961
50962
50962
50963
50963
50964
50964
74709
74709
74710
74710
74711
74711
74712
74712
74713
74713
74714
74714
74715
74715
74716
74716
74717
74717
74718
74718
74719
74719
74720
74720
74721
None
None
0
74721
None
None
0
74722
None
None
0
74722
None
None
0
74723
None
None
0
74723
None
None
0
74725
None
None
0
74725
None
None
0
74727
None
None
0
74727
None
None
0
74728
None
None
0
74728
None
None
0
74729
None
None
0
74729
None
None
0
74731
None
None
0
74731
None
None
0
74732
None
None
0
74732
None
None
0
74733
None
None
0
74733
None
None
0
74736
None
None
0
74736
None

52745
52745
52746
52746
52747
52747
74931
74931
52748
52748
74685
74685
52749
52749
52750
52750
74932
74932
52804
None
None
0
52804
None
None
0
52805
None
None
0
52806
None
None
0
52807
None
None
0
52808
None
None
0
52808
None
None
0
74816
None
None
0
74684
None
None
0
52897
52898
None
<strong>New GE Credit (Start Fall 2011 catalog rights):</strong>
1
52898
None
<strong>New GE Credit (Start Fall 2011 catalog rights):</strong>
1
52898
None
<strong>New GE Credit (Start Fall 2011 catalog rights):</strong>
1
52899
None
<strong>New GE Credit (Start Fall 2011 catalog rights):</strong>
1
52899
None
<strong>New GE Credit (Start Fall 2011 catalog rights):</strong>
1
52899
None
<strong>New GE Credit (Start Fall 2011 catalog rights):</strong>
1
52902
52902
52903
52903
52904
52904
52905
52905
52906
52906
52907
52907
52908
52908
52909
52909
52910
52910
52911
52911
52912
52912
52913
52913
52914
52914
52915
52915
52916
52916
52917
52917
52918
52918
52919
52919
52920
52920
52921
52921
52922
52922
5292

74251
74251
74252
74253
None
None
0
74253
None
None
0
74254
74254
74255
None
None
0
56871
None
None
0
56871
None
None
0
56872
None
None
0
56873
None
None
0
74518
None
None
0
57135
57136
75135
57200
None
None
0
57201
None
None
0
57203
57204
57205
57206
57207
None
None
0
74632
74632
74633
74633
74634
74634
74997
74997
74998
74998
74999
74999
75000
75000
57324
None
None
0
57324
None
None
0
57326
None
None
0
57457
57457
57458
None
<strong>New GE Credit (Start Fall 2011 catalog rights):</strong>
1
74212
57468
57469
57470
74213
74960
74960
74961
74961
74962
74962
74963
74963
74964
74964
74965
74965
57486
57486
57487
57487
57488
57488
74966
74966
74973
74973
75001
75001
75002
75002
75003
75003
75004
75004
75005
75005
75006
75006
75007
75007
75008
75008
75009
75009
75010
75010
57501
57501
57502
57502
57503
57503
57504
57504
57505
57505
57506
57506
57507
57507
57508
57508
57509
57509
57510
57510
57511
57511
57512
57512
57513
57513
57514
57514
57515
57515
75026
75026
75027
75027
75028
75028
5751

74235
None
None
0
62864
None
None
0
74551
None
None
0
75315
None
None
0
75317
None
None
0
75319
None
None
0
75321
None
None
0
62868
None
None
0
62868
None
None
0
62873
None
None
0
62873
None
None
0
74474
None
None
0
74768
None
None
0
62877
62877
62878
62878
62879
62879
62880
62880
62881
62881
62882
62882
62883
62883
62884
62884
62885
62885
62886
62886
62887
62887
62888
62888
75189
75189
75190
75190
75191
75191
75192
75192
74475
None
<strong>New GE Credit (Start Fall 2011 catalog rights):</strong>
1
74475
None
<strong>New GE Credit (Start Fall 2011 catalog rights):</strong>
1
74201
None
None
0
74201
None
None
0
74202
None
None
0
74202
None
None
0
74203
None
None
0
74203
None
None
0
74204
None
None
0
74204
None
None
0
74535
74535
74536
74536
74537
74537
74538
74538
62890
62890
62891
62891
62892
62892
74335
74335
74347
74347
74348
74348
74349
74349
74350
74350
74338
74338
74340
74340
74345
74345
74534
74534
75167
75167
75168
75168
75169
75169
75170
75170
62893
62893
62894
62894
62895
6289

66269
None
None
0
66270
None
None
0
66271
None
None
0
66273
None
None
0
66275
None
None
0
66276
None
None
0
66277
None
None
0
66278
None
None
0
66279
None
None
0
66280
None
None
0
66281
None
None
0
66282
None
None
0
66283
None
None
0
66284
None
None
0
66285
None
None
0
66286
None
None
0
66287
None
None
0
66288
None
None
0
66289
None
None
0
66290
None
None
0
66291
None
None
0
66292
None
None
0
66293
None
None
0
66294
None
None
0
66295
None
None
0
66296
None
None
0
66297
None
None
0
66298
None
None
0
66299
None
None
0
66300
None
None
0
66301
None
None
0
66302
None
None
0
66303
None
None
0
75432
None
None
0
75433
None
None
0
66324
None
None
0
66324
None
None
0
66325
None
None
0
66325
None
None
0
66326
None
None
0
66326
None
None
0
66327
None
None
0
66327
None
None
0
66328
None
None
0
66330
None
None
0
66546
66547
None
None
0
66548
None
None
0
66764
66764
66765
66765
66766
66766
66767
66767
66768
66768
66769
66769
66770
66770
66771
66771
66772
66772
66773
66773
66774
66774
66775
66775
6677

70000
None
<strong>New GE Credit (Start Fall 2011 catalog rights):</strong>
1
70001
None
<strong>New GE Credit (Start Fall 2011 catalog rights):</strong>
1
70001
None
<strong>New GE Credit (Start Fall 2011 catalog rights):</strong>
1
70001
None
<strong>New GE Credit (Start Fall 2011 catalog rights):</strong>
1
70002
None
<strong>New GE Credit (Start Fall 2011 catalog rights):</strong>
1
70002
None
<strong>New GE Credit (Start Fall 2011 catalog rights):</strong>
1
70002
None
<strong>New GE Credit (Start Fall 2011 catalog rights):</strong>
1
70003
None
<strong>New GE Credit (Start Fall 2011 catalog rights):</strong>
1
70003
None
<strong>New GE Credit (Start Fall 2011 catalog rights):</strong>
1
70003
None
<strong>New GE Credit (Start Fall 2011 catalog rights):</strong>
1
70004
None
<strong>New GE Credit (Start Fall 2011 catalog rights):</strong>
1
70004
None
<strong>New GE Credit (Start Fall 2011 catalog rights):</strong>
1
70004
None
<strong>New GE Credit (Start Fall 2011 catalog rights

In [117]:
cou_detail.shape

(13814, 23)

In [118]:
cou_detail.to_csv("course_detail.csv")